# Estudio grasa corporal y obesidad


## Lectura, limpieza, transformación y cálculo

1. Leemos el archivo
2. Limpiamos y eliminamos los datos erróneos
3. Seleccionamos los datos que nos interesan
4. Calculamos algunas variables necesarias para el estudio
5. Guardamos el resultado en un archivo parquet para la continuación del estudio

### Estudio Obesidad

1. Clasificación de obesidad según el BMI:

- Bajo peso (0): BMI < 18.5 
- Peso normal (1): 18.5 ≤ BMI < 25
- Sobrepeso (2): 25 ≤ BMI < 30
- Obesidad (3): BMI ≥ 30

2. Clasificación de riesgo de enfermedades no transmisibles y obesidad según la circunferencia de cintura (CC)

- Si masculino y CC > 94 cm -> Obesidad
- Si femenino y CC > 80 cm -> Obesidad

3. Clasificación de riesgo de enfermedades no transmisibles según el racio circunferencia de cintura y circunferencia de cadera (RCC)

|Riesgo|Femenino|Masculino|
|------|---------|--------|
|Bajo| < 0.8 | < 0.95|
|Medio|0.81 - 0.85 |0.96 - 1 |
|Alto| > 0.86| > 1|

4. Clasificación de riesgo sobrepeso y riesgo de enfermedades no transmisibles según racio circunferencia cintura y talla (ICT):

**ESTO LO CAMBIÉ EN EL CÓDIGO PARA DISMINUIR CATEGORÍAS**

|Riesgo|Femenino|Masculino|
|------|---------|--------|
|Muy delgado| < 0.34 | < 0.34|
|Delgado sano|0.35 - 0.41 |0.35 - 0.42 |
|Sano| 0.42 - 0.48| 0.43 - 0.52|
|Sobrepeso| 0.49 - 0.53 | 0.53 - 0.57|
|Sobrepeso elevado| 0.54 - 0.57| 0.58 a 0.62|
|Obesidad mórbidad| > 0.58| > 0.63|




In [1]:
# Librerías

from pathlib import Path

import pandas as pd
import numpy as np

# Armando pipelines
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [2]:
def read_file(file, sep=','):
    
    df = pd.DataFrame()
      
    file_type = file[file.index('.') - len(file) + 1:]
       
    if (file_type == 'csv'):
        df = pd.read_csv(file, sep=sep)
    elif (file_type == 'parquet'):
        df = pd.read_parquet(file, engine='fastparquet')
    elif (file_type == 'xlsx' or file_type == 'xls'):
        df = pd.read_excel(file)

    return df

In [3]:
# Seleccionar solo las columnas necesarias 

def select_data(df, col_drop=[], col_leave=[]):
     
    if col_leave:
        df = df.loc[:,col_leave]
    
    if col_drop:
        df.drop(col_drop, axis=1, inplace=True)
    
    return df


In [4]:
# Clasificación según BMI: Body Mass Index (Índice de Masa Corporal)

# Cálculo del índice de masa corporal (BMI)
def bmi_calc(weight, height):
    height = height/100
    bmi = weight / (height*height)    
    return bmi

def calc_obesity_bmi(bmi):
    if bmi < 18.5: return 0
    if bmi >= 18.5 and bmi < 25 : return 1
    if bmi >= 25 and bmi < 30 : return 2
    if bmi >= 30 : return 3
    
def calc_obesity_bmi_txt(bmi):
    if bmi == 0: return '0-Bajo Peso'
    if bmi == 1 : return '1-Normal'
    if bmi == 2 : return '2-Sobrepeso'
    if bmi == 3 : return '3-Obesidad'
    

In [5]:
# Clasificación según CC: Circunferencia de cadera

def calc_obesity_cc(gender, waist_circum):
    if gender=='male':
        if waist_circum>94.0: return 1            
    if gender=='female':
        if waist_circum>80.0: return 1 
    return 0

def calc_obesity_cc_txt(cc):
    if cc == 0: return '0-Bajo'
    if cc == 1 : return '1-Alto'


In [6]:
# Clasificación según RCC: Racio entre la circunferencia de la cintura y el de la cadera

def calc_obesity_rcc(gender, rcc):

    if gender == 'female':
        if rcc <= 0.8: return 0
        if rcc > 0.8 and rcc <= 0.85 : return 1
        if rcc > 0.85 : return 2
        
    if gender == 'male':
        if rcc <= 0.95: return 0
        if rcc > 0.95 and rcc <= 1 : return 1
        if rcc > 1 : return 2
        
def calc_obesity_rcc_txt(rcc):
    if rcc == 0: return '0-Bajo'
    if rcc == 1: return '1-Medio'
    if rcc == 2: return '2-Alto'
    


In [7]:
# Clasificación ICT: Índice o racio de circunferencia de cintura y talla (estatura)

def calc_obesity_ict(gender, ict):
    
    if gender == 'female':
        if ict <= 0.41: return 0
        if ict > 0.41 and ict <= 0.48 : return 1
        if ict > 0.48 and ict <= 0.57 : return 2
        if ict > 0.57 : return 3
        
    if gender == 'male':
        if ict <= 0.42: return 0
        if ict > 0.42 and ict <= 0.52 : return 1
        if ict > 0.52 and ict <= 0.62 : return 2
        if ict > 0.62 : return 3
        
def calc_obesity_ict_txt(ict):
    if ict == 0 : return '0-Delgado'
    if ict == 1 : return '1-Normal'
    if ict == 2 : return '2-Sobrepeso'
    if ict == 3 : return '3-Obesidad'
           


In [8]:
# Cuántos factores de riesgo posee una persona

def calc_risk_factors(bmi, cc, rcc, ict):  
    
    total = 0
    
    if bmi >= 2: total = total + 1
    if cc >= 1: total = total + 1
    if rcc >= 1: total = total + 1
    if ict >= 2: total = total + 1
        
    return total

    

In [9]:
# Transforma los datos del dataset, 
# en particular pasa las medidas de pulgadas a cm
# y de libras a Kg
# Esta función puede aplicarse a todo el dataset o a un subconjunto del mismo

def custom_transformations(df):
    
    # Columnas que no están en cm
    not_in_cm = ['age', 'age_range', 'gender', 'weight', 'num_children', 'bra_size_chest', 'bra_size_cup', 'shoe_size_us']
    
    # Columnas en cm: Son todas las columnas menos las que se encuentran más arriba
    in_cm_columns = [ele for ele in df.select_dtypes('float64').columns.to_list() if ele not in not_in_cm]
    
    # Transformamos de pulgadas a cm 
    for col in in_cm_columns:
        df[col] = df.apply(lambda  row: row[col] * 2.54, axis=1)
    
    # Transformamos de libras a Kg
    df['weight'] = df.apply(lambda  row: row['weight'] * 0.454, axis=1)

    return df

In [10]:
# Calcula nuevas variables para el dataset

def custom_calculus(df):
    
    df['gender_bin'] = df.apply(lambda  row: (1 if row['gender']=='male' else 0), axis=1)
    
    # Calculamos el BMI para cada registro
    df['bmi'] = df.apply(lambda  row: bmi_calc(row['weight'], row['height']) , axis=1)

    # Calculamos el racio cintura / cadera para cada registro
    df['rcc'] = df.apply(lambda  row: row['waist_circum_preferred'] / row['hip_circum'], axis=1)
    
    # Calculamos el racio cintura / talla para cada registro
    df['ict'] = df.apply(lambda  row: row['waist_circum_preferred'] / row['height'], axis=1)
    
    df['obesity_bmi'] = df.apply(lambda  row: calc_obesity_bmi(row['bmi']) , axis=1)
    df['obesity_bmi_txt'] = df.apply(lambda  row: calc_obesity_bmi_txt(row['obesity_bmi']) , axis=1)
    
    df['obesity_cc'] = df.apply(lambda  row: calc_obesity_cc(row['gender'], row['waist_circum_preferred']) , 
                                            axis=1)
    df['obesity_cc_txt'] = df.apply(lambda row: calc_obesity_cc_txt(row['obesity_cc']), axis=1)
    
    df['obesity_rcc'] = df.apply(lambda row: calc_obesity_rcc(row['gender'], row['rcc']), axis=1)
    df['obesity_rcc_txt'] = df.apply(lambda row: calc_obesity_rcc_txt(row['obesity_rcc']), axis=1)
    
    
    df['obesity_ict'] = df.apply(lambda row: calc_obesity_ict(row['gender'], row['ict']) , axis=1)
    df['obesity_ict_txt'] = df.apply(lambda row: calc_obesity_ict_txt(row['obesity_ict']) , axis=1)

    
    df['risk_factors'] = df.apply(lambda  row: calc_risk_factors(row['obesity_bmi'], 
                                                                   row['obesity_cc'], 
                                                                   row['obesity_rcc'],
                                                                   row['obesity_ict']) , 
                                            axis=1)
    
    return df

In [11]:
# Permite hacer transformaciones y cálculos personalizados al subconjunto de datos
# Es un pipeline que llama 2 funciones custom_transformations y custom_calculus

def transform_calcul(df):
    # Aplicamos transformaciones y cálculos personalizados
    custom_transformations_pipeline = Pipeline([
        ('custom_transforms', FunctionTransformer(custom_transformations)), # Función de transformación personalizada
        ('custom_calculus', FunctionTransformer(custom_calculus)) # Función de cálculos personalizados
    ])

    # Aplicar el pipeline al conjunto de datos
    return custom_transformations_pipeline.fit_transform(df)

In [12]:
# Esta función puede ser utilizada para extraer un subconjunto de datos del dataset original
# Parámetros:
#   columns_to_drop: Lista de columnas que se quiere borrar (cuando se desea conservar la mayoría)
#   columns_to_leave: Lista de columnas que se quiere conservar
#   transform: Boolean que indica si se va o no ejecutar la función transform_calculEs

def load_clean_transform(columns_to_drop, columns_to_leave, transform=True):
    
    import pickle
    
    # Leemos el archivo
    CURRENT_DIR = Path.cwd()
    
    BASE_DIR = Path(CURRENT_DIR).parent

    file = f"{BASE_DIR}/data/in/caesar.csv"

    df = read_file(file, ',')

    # Limpiamos y eliminamos los datos erróneos
    
    # Borramos la primera línea que no contiene datos significativos
    df.drop([0], axis=0, inplace=True)
    
    # Seleccionamos las variables que nos interesan
    # Esta función debería hacer un select en la BD para traer lo que nos interesa
    # según el problema que estamos tratando
    df = select_data(df, columns_to_drop, columns_to_leave)

    # Si la cantidad de registros con valores nulos es menos de 1% los eliminamos
    nul_vals = df[ df.isnull().values ]['gender'].count() / df['gender'].count()

    if nul_vals <= 0.01: df.dropna(inplace=True)
        
    if transform:
        #transform_pipeline = transform_calcul(df)
        
        df = transform_calcul(df)
        
        # save the model to a file
        #with open(f"{BASE_DIR}/models/transform_pipeline.pkl", "wb") as f:
        #    pickle.dump(transform_pipeline, f)
            
        #df = transform_pipeline.transform(df)
    
    
    # Guardamos el resultado en un archivo parquet para el resto del estudio
    
    df.to_parquet(f"{BASE_DIR}/data/out/obesity.parquet", 
                    compression='GZIP',
                    engine='pyarrow')
    
    return df

## Main

In [13]:
columns_to_drop = []
columns_to_leave = [
 'age',
 'age_range',
 'gender',
 'height',
 'weight',
 'waist_circum_preferred',
 'hip_circum'
]

df_obesity = load_clean_transform(columns_to_drop, columns_to_leave, True)

In [14]:
df_obesity

,age,age_range,gender,height,weight,waist_circum_preferred,hip_circum,gender_bin,bmi,rcc,ict,obesity_bmi,obesity_bmi_txt,obesity_cc,obesity_cc_txt,obesity_rcc,obesity_rcc_txt,obesity_ict,obesity_ict_txt,risk_factors
1,47.0,46-55,male,187.1980,156.630,138.5062,142.1892,1,44.696399,0.974098,0.739891,3,3-Obesidad,1,1-Alto,1,1-Medio,3,3-Obesidad,4
2,50.0,46-55,male,180.5940,98.064,98.3996,109.9058,1,30.067891,0.895309,0.544866,3,3-Obesidad,1,1-Alto,0,0-Bajo,2,2-Sobrepeso,3
3,28.0,26-35,male,182.7022,89.211,97.0026,107.0102,1,26.725809,0.906480,0.530933,2,2-Sobrepeso,1,1-Alto,0,0-Bajo,2,2-Sobrepeso,3
4,52.0,46-55,male,185.9026,113.273,105.9942,111.0996,1,32.775964,0.954047,0.570160,3,3-Obesidad,1,1-Alto,1,1-Medio,2,2-Sobrepeso,4
5,50.0,46-55,male,183.1086,108.279,104.0892,112.3950,1,32.294365,0.926102,0.568456,3,3-Obesidad,1,1-Alto,0,0-Bajo,2,2-Sobrepeso,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,35.0,26-35,female,152.8064,49.259,61.5950,93.0910,0,21.096116,0.661664,0.403092,1,1-Normal,0,0-Bajo,0,0-Bajo,0,0-Delgado,0
4461,40.0,36-45,female,168.6052,67.646,73.8124,100.4062,0,23.795794,0.735138,0.437782,1,1-Normal,0,0-Bajo,0,0-Bajo,1,1-Normal,0
4462,23.0,17-25,female,168.5036,73.775,78.0034,109.7026,0,25.983093,0.711044,0.462918,2,2-Sobrepeso,0,0-Bajo,0,0-Bajo,1,1-Normal,1
4463,24.0,17-25,male,170.5102,65.149,75.7936,97.7900,1,22.408203,0.775065,0.444511,1,1-Normal,0,0-Bajo,0,0-Bajo,1,1-Normal,0


In [15]:
df_obesity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4430 entries, 1 to 4464
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     4430 non-null   float64
 1   age_range               4430 non-null   object 
 2   gender                  4430 non-null   object 
 3   height                  4430 non-null   float64
 4   weight                  4430 non-null   float64
 5   waist_circum_preferred  4430 non-null   float64
 6   hip_circum              4430 non-null   float64
 7   gender_bin              4430 non-null   int64  
 8   bmi                     4430 non-null   float64
 9   rcc                     4430 non-null   float64
 10  ict                     4430 non-null   float64
 11  obesity_bmi             4430 non-null   int64  
 12  obesity_bmi_txt         4430 non-null   object 
 13  obesity_cc              4430 non-null   int64  
 14  obesity_cc_txt          4430 non-null   

In [16]:
df_obesity.describe()

,age,height,weight,waist_circum_preferred,hip_circum,gender_bin,bmi,rcc,ict,obesity_bmi,obesity_cc,obesity_rcc,obesity_ict,risk_factors
count,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000
mean,37.764312,170.937562,75.205328,84.701265,103.513457,0.471783,25.608337,0.816921,0.495404,1.598871,0.345824,0.272009,1.403837,1.383521
std,12.786156,10.387864,18.419132,13.362246,10.273928,0.499260,5.288433,0.085733,0.071842,0.777331,0.475690,0.589520,0.765085,1.525385
min,17.500000,124.790200,37.332420,55.702200,80.010000,0.000000,15.263630,0.583295,0.334210,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.400000,163.322000,61.957380,75.006200,96.697800,0.000000,21.940406,0.752614,0.445829,1.000000,0.000000,0.000000,1.000000,0.000000
50%,37.000000,170.408600,72.186000,83.007200,101.904800,0.000000,24.498035,0.820929,0.482352,1.000000,0.000000,0.000000,1.000000,1.000000
75%,48.000000,177.901600,84.898000,92.278200,108.102400,1.000000,27.973753,0.876377,0.530471,2.000000,1.000000,0.000000,2.000000,3.000000
max,79.000000,218.287600,181.600000,170.205400,183.388000,1.000000,57.180160,1.153405,0.970878,3.000000,1.000000,2.000000,3.000000,4.000000
